## Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  

## Equipe:

1. Afonso Henrique Torres Lucas | ifonso.developer@gmail.com | iFonso - 2215080047
2. Emanuelle Rocha Marreira | emanuellemarreira@gmail.com | emanuellemarreira - 2215080035
3. Erik Gustavo Lima de Oliveira | erik.exatas10@gmail.com | ErikExatas - 2115080049
4. David Augusto De Oliveira E Silva | david-augusto-silva - 2115080006
5. Ítalo Ferreira Fonseca
6. João Vitor Silva De Carvalho
7. Lilian Iazzai De Souza Oliveira | lilianiazzai@gmail.com | lilianiazzai - 2215080018
8. Vitor Nascimento Aguiar | Vtaguiar1909 - 2115080055

- **Github**: https://github.com/emanuellemarreira/image-authenticity-classification/tree/parte1

In [1]:
# bibliotecas
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader, ConcatDataset
import numpy as np
import kagglehub
from collections import Counter
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, Subset
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

**Descrição da Atividade**

A tarefa de aprendizado de máquina definida neste trabalho é a classificação de imagens binária, com o objetivo de identificar se uma imagem é real (fotografia de pessoas reais) ou sintética, ou seja, gerada por Inteligência Artificial Utilizamos o dataset CIFAKE: Real and AI-Generated Synthetic Images, disponível no Kaggle, que contém imagens divididas em duas classes: "real" e "fake". A classe "real" é composta por fotografias autênticas de rostos humanos, enquanto a classe "fake" contém imagens de rostos geradas por algoritmos de IA, como GANs (Redes Generativas Adversariais). Como se trata de um problema visual e a tarefa envolve reconhecer padrões sutis na textura, iluminação e traços faciais que podem diferenciar uma imagem real de uma sintética, optamos por utilizar uma Rede Neural Convolucional (CNN), que é uma arquitetura especialmente eficaz para extração automática de características em tarefas de classificação de imagens.

Link do dataset: https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images


### 1. Leitura do dataset

In [2]:
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

In [3]:
path

'/kaggle/input/cifake-real-and-ai-generated-synthetic-images'

In [4]:
#dataset_path = path
train_path = f"{path}/cifake/train"
test_path = f"{path}/cifake/test"

In [5]:
transform = transforms.Compose([
    # Resize da imagem pra 32x32 (pixels), seria bom colocar 128x128
    transforms.Resize((32, 32)),
    # PIL -> Tensor[canal, altura, largura] + Normalização de pixels
    transforms.ToTensor()
])

In [7]:
dataset = ImageFolder(root=path, transform=transform)

In [8]:
train_data = datasets.ImageFolder(root=f"{path}/train", transform=transform)
test_data = datasets.ImageFolder(root=f"{path}/test", transform=transform)

In [9]:
dataset

Dataset ImageFolder
    Number of datapoints: 120000
    Root location: /kaggle/input/cifake-real-and-ai-generated-synthetic-images
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

### 2. Mini Análise Exploratória

In [10]:
train_labels = train_data.targets

In [11]:
train_label_counts = Counter(train_labels)

In [12]:
train_label_counts # 0: FAKE / 1: REAL

Counter({0: 50000, 1: 50000})

In [13]:
train_data.classes

['FAKE', 'REAL']

O conjunto de dados de treino já está balanceado, contendo 50.000 imagens reais (classe 1) e 50.000 imagens geradas por IA (classe 0). Isso significa que o modelo será treinado com a mesma quantidade de exemplos para cada classe, o que ajuda a evitar viés de aprendizado. Com esse equilíbrio, a CNN tem maiores chances de aprender a distinguir padrões relevantes de cada classe de forma justa, sem favorecer uma categoria em detrimento da outra, o que pode resultar em melhor desempenho e métricas mais confiáveis.


### 3. Holdout estratificado: 70% treino / 30% validação
Neste trabalho, optamos por utilizar a validação do tipo holdout estratificado, dividindo o conjunto de dados em 70% para treino, 15% para validação e 15% para teste. Embora a validação cruzada k-fold ofereça uma avaliação mais robusta por repetir o treinamento múltiplas vezes com diferentes divisões dos dados, ela também apresenta um custo computacional significativamente maior — especialmente quando se utiliza redes neurais convolucionais (CNNs) e um volume elevado de dados, como no caso deste projeto. O holdout, por sua vez, permite uma separação mais simples e rápida, com desempenho adequado quando há uma boa quantidade de dados e as classes estão balanceadas. Por isso, a escolha pelo holdout foi feita visando um equilíbrio entre qualidade de avaliação e viabilidade computacional.

In [14]:
train_labels = train_data.targets

In [15]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_labels)), train_labels))

In [16]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [17]:
print(f"Treino: {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste: {len(test_data)} imagens")

Treino: 70000 imagens
Validação: 30000 imagens
Teste: 20000 imagens
